# Foundation models for zero-shot detection and segmentation

Based on [Ollama](https://github.com/ollama/ollama) project.

## Preparing Python tools

In [ ]:
!pip install supervision

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
import json
import base64
import requests
import torch
import PIL
from transformers import AutoProcessor, AutoModelForZeroShotObjectDetection
import cv2
import supervision
import numpy
import datetime
import logging
import copy


logger = logging.getLogger(__name__)
__detection_models = {}


def get_model(name: str):
    if not name in __detection_models:
        __detection_models[name] = ZShotModel(model=name)
    return __detection_models[name]


class ZShotModel:
    def __init__(self, model: str):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model=AutoModelForZeroShotObjectDetection.from_pretrained(
            model).to(self.device)
        self.processor=AutoProcessor.from_pretrained(model)

    def infer(self, images: PIL.Image.Image | list[PIL.Image.Image],
              prompt: str | list[str],
              box_threshold: float=0.2,
              text_threshold: float=0.2):
        # VERY important: text queries need to be lowercased + end with a dot
        if isinstance(prompt, list):
            objects = [item for item in prompt if not item=='']
            text = " . ".join([f"{item}" for item in objects]).lower() + "."
        elif isinstance(prompt, str):
            text = prompt.lower() + "."
        else:
            raise ValueError(
                f"Error, prompt with type \"{type(prompt)}\" is not supported")
        inputs = self.processor(
            images=images, text=text, return_tensors="pt"
        ).to(self.device)
        with torch.no_grad():
            outputs = self.model(**inputs)

        results = self.processor.post_process_grounded_object_detection(
            outputs,
            inputs.input_ids,
            box_threshold=box_threshold,
            text_threshold=text_threshold,
            target_sizes=[image.size[::-1]]
        )

        labels = results[0]["labels"]
        class2label = list(set(labels))
        label2class = {
            class2label[i]: i for i in range(0, len(class2label))
        }
        classes = [label2class[label] for label in labels]
        results[0]["class2label"] = class2label
        results[0]["label2class"] = label2class
        results[0]["label_classes"] = classes

        return results[0]


def load_answer(filename: str) -> dict[str, any]:
    with open(filename, "r") as file:
        step2_response = json.loads(file.read())
    return step2_response["response"] if "response" in step2_response else step2_response


def llm_infer(model: str,
              prompt: str,
              image_file: str | list[str] | None=None,
              log_pattern: str | None=None,
              url: str = "http://localhost:11434/api/generate"
              ) -> dict[str, any]:
    payload = dict(model=model, prompt=prompt, stream=False)
    if image_file is not None:
        if isinstance(image_file, str):
            image_file = [image_file]
        images = []
        for image in image_file:
            with open(image, "rb") as file:
                encoded_image = base64.b64encode(file.read()).decode("ascii")
            images.append(encoded_image)
        payload["images"] = images

    if log_pattern is not None:
        with open(f"{log_pattern}_request.json", "w") as file:
            file.write(json.dumps(payload))

    reply = requests.post(url, json=payload)

    if log_pattern is not None:
        with open(f"{log_pattern}_reply.json", "w") as file:
            file.write(reply.content.decode("ascii"))


    return json.loads(reply.content.decode("ascii"))


def text_to_objects(
        text: str,
        start: str="OUTPUT",
        end: str="INFO",
        rem: str=":*") -> list[str]:
    clean_list = text
    for char in rem:
        clean_list = clean_list.replace(char, '')

    split1 = clean_list.split(start)
    if len(split1) < 2:
        return []
    split2 = split1[1].split(end)
    if len(split2) < 2:
        return []
    objects = list(set(split2[0].split("\n")))
    if '' in objects:
        objects.remove('')

    return objects


def bbox_image(image: PIL.Image.Image, bbox: dict[str, any]) -> None:
    box_annotator = supervision.BoxAnnotator()
    label_annotator = supervision.LabelAnnotator()

    image_boxes = supervision.Detections(
        xyxy=bbox["boxes"].cpu().numpy(),
        class_id=numpy.array(bbox["label_classes"], dtype=int)
    )

    #, 2, 3, 4])#results[0]["labels"]
    """
    labels = [
        f"{class_id} {confidence:0.2f}"
        for confidence, class_id, boxes in results
    ]
    """
    annotated_frame = box_annotator.annotate(
        scene=image.copy(), detections=image_boxes) #, labels=labels)
    annotated_frame = label_annotator.annotate(
        scene=annotated_frame,
        detections=image_boxes,
        labels=bbox["labels"]
    )
    supervision.plot_image(annotated_frame, (16, 16))


def detect_all_objects(
        image: str,
        captioning_prompt: str,
        object_extraction_prompt: str,
        bbox_threshold: float=0.1,
        text_threshold: float=0.1,
        captioning_model: str="llava",
        parsing_model: str="llama3.1",
        detection_model: str="IDEA-Research/grounding-dino-base",
        start_objects_list_marker: str="OUTPUT",
        end_objects_list_marker: str="INFO",
        symbols_to_remove: str="*:",
        capturing_iterations: int=1,
        parsing_iterations: int=1
) -> dict[str, any]:
    calculation_id = (
        captioning_model + "_" + datetime.datetime.utcnow().strftime(
            "%Y-%m-%d_%H.%M.%S_%f")
    )

    llava_reply = []
    for i in range(0, capturing_iterations):
        try:
            llava_reply.append(llm_infer(
                    model=captioning_model,
                    prompt=captioning_prompt,
                    image_file=image,
                    log_pattern=captioning_model + "_" + calculation_id
                ))
        except Exception as e:
            logger.error("Capturing reply error for %s, skipping",
                            parsing_model + "_" + calculation_id,
                            exc_info=e)
            continue

    objects = set()
    for j in range(0, parsing_iterations):
        for i in range(0, capturing_iterations):
            try:
                llama_reply = llm_infer(
                    model=parsing_model,
                    prompt=object_extraction_prompt.format(
                        llava_reply[i]["response"]),
                    log_pattern=parsing_model + "_" + calculation_id
                )
                objects = objects | set(text_to_objects(
                    text=llama_reply["response"],
                    start=start_objects_list_marker,
                    end=end_objects_list_marker,
                    rem=symbols_to_remove))
            except Exception as e:
                logger.error("Processing reply error for %s, skipping",
                             parsing_model + "_" + calculation_id,
                             exc_info=e)
                continue

    image_data = PIL.Image.open(image)

    detected_objects = get_model(
        name=detection_model).infer(
            images=image_data,
            prompt=list(objects),
            box_threshold=bbox_threshold,
            text_threshold=text_threshold
        )

    return detected_objects


def save_detection_results(detection_results: dict[str, any], filename: str) -> None:
    data = copy.deepcopy(detection_results)
    data["boxes"] = data["boxes"].tolist()
    data["scores"] = data["scores"].tolist()
    with open(filename, "w") as file:
        file.write(json.dumps(data, sort_keys=True, indent=True))


## Installing Ollama and pulling LLMs

In [ ]:
!wget -O ollama.tgz https://ollama.com/download/ollama-linux-amd64.tgz

In [ ]:
!tar -xvzf ollama.tgz

In [ ]:
# !curl -L https://ollama.com/download/ollama-linux-amd64 -o ollama
# !chmod +x ollama

In [ ]:
import subprocess


subprocess.Popen(["./bin/ollama", "serve"])
import time
time.sleep(3)

In [ ]:
!./bin/ollama pull llava
!./bin/ollama pull llama3.1

In [ ]:
import requests


result = requests.post("http://localhost:11434/api/generate", json={
        "model": "llama3.1",
        "prompt": "Why do you cry",
        "stream": False
    })
(result.status_code, result.content)

## Experiment #1

In [ ]:
!wget -q -O image.jpg https://github.com/ant-nik/neural_network_course/blob/main/practice_2_data/video_1_fixed/image_001.jpg?raw=true

In [ ]:
llava_prompt = "Describe entities on the image as detailed as possible."
llama_prompt = """Extract all nouns from the TEXT section that are physical objects, living beings, dressing, parts of living beings or physical objects.
Split answer in two parts: OUTPUT and INFO.
In OUTPUT section place extracted nouns without enumerations symbols and one entity per line.
Put detailed explanation of the answer to INFO section.

TEXT:
{}

OUTPUT:
"""

In [ ]:
%matplotlib inline
image = PIL.Image.open("/content/image.jpg")
detected_objects = detect_all_objects(
        image="/content/image.jpg",
        captioning_prompt=llava_prompt,
        object_extraction_prompt=llama_prompt,
        capturing_iterations=2,
        parsing_iterations=2
        )
bbox_image(image=image, bbox=detected_objects)

In [ ]:
save_detection_results(detected_objects, "results.json")

In [ ]:
# print(load_answer("/content/llama3.1_llava_2024-08-23_14.35.15_810177_request.json")["prompt"])

# Objects count by confidence score thresholds

In [ ]:
all_results = detect_all_objects(
        image="/content/image.jpg",
        captioning_prompt=llava_prompt,
        object_extraction_prompt=llama_prompt,
        bbox_threshold=0.01,
        text_threshold=0.01
)

In [ ]:
len(all_results["class2label"])

In [ ]:
x = numpy.linspace(0.01, 1, 100)
y = numpy.diff([len([x for x in filter(lambda x: x > threshold, all_results["scores"])]) for threshold in x])

In [ ]:
import plotly.express


plotly.express.line(x=x[1:], y=y)

In [ ]:
plotly.express.histogram(y)

In [ ]:
numpy.quantile(y, [0.01, 0.05, 0.1, 0.15, 0.2])

In [ ]:
filtered = detect_all_objects(
        image="/content/image.jpg",
        captioning_prompt=llava_prompt,
        object_extraction_prompt=llama_prompt,
        bbox_threshold=0.08,
        text_threshold=0.08
)
(len(filtered["labels"]), filtered["class2label"])

## Detected entities count vs LLM's iterations

In [ ]:
llava_attemps = 3
llama_attemps = 3
averaging_attemps = 3

objects_stat = numpy.ndarray(
    shape=(llava_attemps, llama_attemps, averaging_attemps, 2),
    dtype=float)
objects_labels = {}
detections = {}
for i in range(0, llava_attemps):
    objects_labels[i] = {}
    detections[i] = {}
    for j in range(0, llama_attemps):
        objects_labels[i][j] = {}
        detections[i][j] = {}
        for k in range(0, averaging_attemps):
            try:
                detected = detect_all_objects(
                        image="/content/image.jpg",
                        captioning_prompt=llava_prompt,
                        object_extraction_prompt=llama_prompt,
                        bbox_threshold=0.08,
                        text_threshold=0.08,
                        capturing_iterations=i + 1,
                        parsing_iterations=j + 1
                )
            except Exception as e:
                print(e)
                objects_stat[i, j, k, 0] = numpy.nan
                objects_stat[i, j, k, 1] = numpy.nan
                objects_labels[i][j][k] = None
                continue

            objects_stat[i, j, k, 0] = len(detected["class2label"])
            objects_stat[i, j, k, 1] = len(detected["labels"])
            objects_labels[i][j][k] = detected["labels"]
            detections[i][j][k] = detected

In [ ]:
import pandas


data = {
    "llava_iter": [], "llama_iter": [], "averaging_index": [],
    "len(classes)": [], "len(labels)": []
}
for i in range(0, objects_stat.shape[0]):
    for j in range(0, objects_stat.shape[1]):
        for k in range(0, objects_stat.shape[2]):
            data["llava_iter"].append(i + 1)
            data["llama_iter"].append(j + 1)
            data["averaging_index"].append(k)
            data["len(classes)"].append(objects_stat[i, j, k, 0])
            data["len(labels)"].append(objects_stat[i, j, k, 1])

dataframe = pandas.DataFrame(data)
dataframe.to_csv("iterations_stat.csv")

with open("full_detection.json", "w") as stat:
    stat.write(str(detections))

In [ ]:
len_classes_arr = numpy.ndarray(
    shape=(llava_attemps, llama_attemps), dtype=float
)

len_labels_arr = numpy.ndarray(
    shape=(llava_attemps, llama_attemps), dtype=float
)

groups = dataframe.groupby(by=["llava_iter", "llama_iter"])
for index, row in groups.mean().iterrows():
    ind = (index[0] - 1, index[1] - 1)
    len_classes_arr[ind] = row["len(classes)"]
    len_labels_arr[ind] = row["len(labels)"]

In [ ]:
import plotly.graph_objects


fig = plotly.graph_objects.Figure(
    data=[
        plotly.graph_objects.Surface(
            z=len_classes_arr),
        plotly.graph_objects.Surface(
            z=len_labels_arr)
    ])
fig.update_layout(title="Mean metrics vs (x=llava, y=llama) iterations")
fig.show()

# Reducing duplicated text

In [ ]:
for